In [105]:
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as scp
random.seed(1)


Exercise 4 Exercise 4
Write a discrete event simulation program for a blocking system,
i.e. a system with m service units and no waiting room. The offered
traffic A is the product of the mean arrival rate and the mean
service time.
1. The arrival process is modelled as a Poisson process. Report the
fraction of blocked customers, and a confidence interval for this
fraction. Choose the service time distribution as exponential.
Parameters: m = 10, mean service time = 8 time units, mean
time between customers = 1 time unit (corresponding to an
offered traffic of 8 Erlang), 10 x 10.000 customers.
This system is sufficiently simple such that the analytical
solution is known. See the last slide for the solution. Verify your
simulation program using this knowledge. 
2. The arrival process is modelled as a renewal process using the
same parameters as in Part 1 when possible. Report the
fraction of blocked customers, and a confidence interval for this
fraction for at least the following two cases
(a) Experiment with Erlang distributed inter arrival times The
Erlang distribution should have a mean of 1
(b) hyper exponential inter arrival times. The parameters for
the hyper exponential distribution should be
p1 = 0.8, λ1 = 0.8333, p2 = 0.2, λ2 = 5.0.
3. The arrival process is again a Poisson process like in Part 1.
Experiment with different service time distributions with the
same mean service time and m as in Part 1 and Part 2.
(a) Constant service time
(b) Pareto distributed service times with at least k = 1.05 and
k = 2.05. 
(c) Choose one or two other distributions.
4. Compare confidence intervals for Parts 1, 2, and 3 then
interpret and explain differences if any.

In [8]:
#define variables: 
m = 10 
mean_service_time = 8
mean_time_between_customer = 1
lambA = 1/mean_time_between_customer
lambS = 1/mean_service_time

In [64]:
def St():
    U1 = np.random.uniform(0,1)
    S = -np.log(U1)/lambS
    return S 
    

In [61]:
def At():
    U1 = np.random.uniform(0,1)
    A=-np.log(U1)/lambA
    return A

In [168]:
def sim(At,St):
    t = 0
    event_list = []
    count_no_service = 0 
    count_service = 0
    m = 10
    total_costumer = 0
    
    while total_costumer<=10000:

        t = t+At()
        service_time = St()

        for i,time in enumerate(event_list):
            if t >= time:
                event_list.pop(i)
                m=m+1

        if m == 0: 
            count_no_service = count_no_service + 1 
            
        else:
            event_list.append(t+service_time)
            count_service = count_service+1
            m = m-1
        total_costumer = total_costumer+1
        
    return count_no_service/total_costumer

In [252]:
#Confidence interval
def conf(ratio,n):
    theta = np.mean(ratio)
    theta2 = theta**2
    ratio2 = np.zeros(len(ratio))

    for i,num in enumerate(ratio):
        ratio2[i] = num**2
    ratio2 = np.sum(ratio2)

    S = 1/(n-1)*(ratio2-n*theta2)
    print(S)
    t0 = scp.norm.ppf(0.975)
    # Confidence interval: 
    return [theta + np.sqrt(S)/np.sqrt(n)*t0, theta - np.sqrt(S)/np.sqrt(n)*t0]

In [253]:
# simulation: 
ratio = []
n=10
for i in range(n):
    ratio.append(sim(At,St))

conf(ratio,n)


2.453164898044702e-05


[0.12476763997268159, 0.11862802046127502]

In [127]:
# Erling distribution impementation: 
# lambda og n skal være lig hinaanden for at opnå mean på 1 da mean(Y) = n/lambda
def AtE():
    n = 1
    U1 = np.random.uniform(0,1,n)
    A = -1/lambA*np.log(np.product(U1))
    return A

In [158]:
ratioErling = []
n=10
for i in range(n): 
    ratioErling.append(sim(AtE,St))
conf(ratioErling,n)

7.369959267779996e-05


[0.12837413452942523, 0.11842118593852798]

In [170]:
#Exponentiel: 
#Vigitgt independent random variabler for lambda og for A
def AtEx():
    p =[0.8, 0.2]
    lamb = [0.8333, 5]

    U1 = np.random.uniform(0,1)
    U2 = np.random.uniform(0,1)
    if U1 <p[0]:
        I=1
    else:
        I=2
    A = - 1/lamb[I-1]*np.log(U2)
    return A



In [172]:
ratioExp = []
n=10
for i in range(n): 
    ratioExp.append(sim(AtEx,St))
conf(ratioExp,n)


2.4716834163773793e-05


[0.1446877641187911, 0.13892387471732529]

In [180]:
def StC():
    A=8
    return A

In [181]:
# pointe laver interval (1 event med variance ikke 2)
ratioStC = []
for i in range(n): 
    ratioStC.append(sim(At,StC))
conf(ratioStC,n)



1.9957008398753375e-05


[0.1255873256019096, 0.1204080748580444]

In [247]:
# Perato: 
def StP():
    k = 1.05
    B = 8 * (k-1) / k 
    U1 = np.random.uniform(0,1)
    A = B *(U1**(-1/k))
    return A 

In [249]:
# Perato confidence interval
ratioStP = []
for i in range(n): 
    ratioStP.append(sim(At,StP))
conf(ratioStP,n)

1.204470205025404e-05


[0.003871625359678251, -0.0001519973224819698]

In [210]:
def StU():
    dist = 7
    m=8
    a = m-dist
    b = m+dist

    U1 = np.random.uniform(0,1)
    A = a+(b-a)*U1
    return A


In [211]:
# Uniform  S(t) confidence interval:  
ratioStU = []
for i in range(n): 
    ratioStU.append(sim(At,StU))
conf(ratioStU,n)


2.4924903658907205e-05


[0.12367197177775023, 0.11788387263780818]

In [245]:
# Normalfordeling: 
def StN():
  
    U1 = np.random.uniform(0,1)
    U2 = np.random.uniform(0,1)
    R = np.sqrt(-2 * np.log(U1))
    theta = 2 * np.pi * U2

    A = R * np.sin(theta)+8
    return A

In [246]:
# Uniform  S(t) confidence interval:  
ratioStN= []
for i in range(n): 
    ratioStN.append(sim(At,StN))
conf(ratioStN,n)


2.268446288058692e-05


[0.12617857600209942, 0.1206567404662537]

In [166]:
sum=[]
for i in range(10000):
    sum.append(AtEx())
mean= np.mean(sum)
print(mean)
print(np.var(sum))

sum1=[]
for i in range(10000):
    sum1.append(AtE())
mean1= np.mean(sum1)
print(mean1)
print(np.var(sum1))

sum2=[]
for i in range(10000):
    sum2.append(At())
mean2= np.mean(sum2)
print(mean2)
print(np.var(sum2))

1.1996026988397481
1.5555088988305221
1.0080130126582616
1.0306772839537564
0.9909463016227106
1.0001962223561611
